## **🧠 Privacy-Preserving Mental Health Risk Detection**

This demo project demonstrates a lightweight privacy-preserving AI system for detecting early mental health risk from short text messages using encrypted inference. We simulate a realistic healthcare scenario where patient-generated data must remain confidential—yet still usable for AI-powered triage. By combining sentence embeddings, classical machine learning, and Fully Homomorphic Encryption (FHE), we enable secure inference on encrypted inputs without revealing sensitive text.

**🔐 Key Features**

**Federated Learning Scenario**: Model is trained locally; inference is performed securely on encrypted user inputs—ideal for settings with distributed, sensitive healthcare data.


**Privacy-Preserving AI**: Raw user data never leaves the client side unencrypted—computation and risk scoring occur securely in ciphertext space.

**Sentence Embeddings**: Uses all-MiniLM-L6-v2 to convert input text into dense semantic vectors.

**Encrypted Inference with TenSEAL**: Applies the CKKS scheme to run logistic regression on encrypted embeddings.


In [ ]:
!pip install sentence-transformers tenseal

**🧮 Introduction to CKKS and TenSEAL**

To enable privacy-preserving inference over sensitive clinical text, this project leverages the CKKS (Cheon-Kim-Kim-Song) scheme for approximate homomorphic encryption. Unlike traditional encryption, CKKS supports arithmetic directly on encrypted real numbers, making it ideal for machine learning workflows involving floating-point operations like dot products and linear models.

We use [TenSEAL](https://github.com/OpenMined/TenSEAL) — a Python library built on top of Microsoft SEAL — to:



*   Encrypt high-dimensional sentence embeddings (e.g., from MiniLM)
*   Perform encrypted linear inference (e.g., logistic regression)
*   Decrypt only the final result, preserving end-to-end confidentiality
*   By operating entirely on ciphertexts, TenSEAL allows computations to be outsourced to untrusted environments (e.g., cloud or remote nodes) without revealing inputs, model parameters, or intermediate results.











In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sentence_transformers import SentenceTransformer

# 1. Simulate 400 labeled text messages
mental_health_texts = [
    "I feel hopeless and tired all the time.",
    "Lately, I can't concentrate and everything feels overwhelming.",
    "I barely talk to anyone and feel isolated.",
    "School is stressing me out beyond what I can handle.",
    "I have no energy to do anything, even things I used to enjoy.",
    "I feel anxious constantly, even when nothing is wrong.",
    "I'm not sleeping well and my appetite is gone.",
    "Everything feels meaningless and I just want to be left alone.",
    "I cry randomly and can't explain why.",
    "Even getting out of bed feels like a chore."
] * 20  # 200 samples

no_problem_texts = [
    "I’ve been sleeping well and enjoying my time with friends.",
    "I feel confident and motivated about my goals.",
    "I’ve been going for daily walks and eating healthy.",
    "Things at school are busy but manageable.",
    "I enjoy socializing and staying active.",
    "Life has been stable and I’m feeling grateful.",
    "I’ve been productive and focused lately.",
    "My energy levels are good and I feel optimistic.",
    "I’ve been taking care of myself and feeling balanced.",
    "Everything is going smoothly and I’m content."
] * 20  # 200 samples

texts = mental_health_texts + no_problem_texts
labels = [1]*200 + [0]*200

df = pd.DataFrame({'text': texts, 'mental_health_problem': labels})
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 2. Encode messages into embeddings
encoder = SentenceTransformer("all-MiniLM-L6-v2")
X_embeddings = encoder.encode(df['text'].tolist())
y = df['mental_health_problem'].values

# 3. Split and train
X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y, test_size=0.2, stratify=y, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 4. Evaluate
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       1.00      1.00      1.00        40

    accuracy                           1.00        80
   macro avg       1.00      1.00      1.00        80
weighted avg       1.00      1.00      1.00        80



**🔓 Plaintext Inference Version**

This simplified version demonstrates the same mental health risk detection pipeline without encryption, serving as a baseline for comparison.

*   Text messages are embedded using a pretrained MiniLM model.
*   A logistic regression classifier predicts mental health risk based on those embeddings.
*   Inference is performed directly on plaintext vectors using a standard dot product.
*   Useful for validating model performance before deploying privacy-preserving encrypted inference.


In [14]:
# 5. Predict on new input (plaintext inference)
test_text = "I feel hopeless and tired all the time."
embedding = encoder.encode([test_text])[0]
score = np.dot(model.coef_[0], embedding) + model.intercept_[0]
probability = 1 / (1 + np.exp(-score))

print(f"\nPrediction probability for:\n\"{test_text}\"\n→ {probability:.4f}")
print("✅ Risk Detected" if probability > 0.5 else "✅ No Risk Detected")



Prediction probability for:
"I feel hopeless and tired all the time."
→ 0.9036
✅ Risk Detected


**🔐 Why Encryption Matters in Clinical AI: Protecting Against Embedding Leakage**



*   🔓 Note: In clinical environments, using plaintext or even unencrypted embeddings is generally not permitted, as embeddings—though not directly human-readable—can still be vulnerable to inversion or re-identification attacks.








In [27]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. Load encoder and corpus of possible texts
encoder = SentenceTransformer("all-MiniLM-L6-v2")
corpus_sentences = [
    "I feel tired and sad all the time.",
    "I'm excited about my new project.",
    "Everything feels meaningless.",
    "I can't concentrate on anything lately.",
    "I’ve been sleeping well and eating healthy.",
    "Life is overwhelming and I want to cry.",
    "My motivation is gone and I feel hopeless."
]

# 2. Encode known corpus of candidate sentences
corpus_embeddings = encoder.encode(corpus_sentences)

# 3. Simulate intercepted embedding (e.g., from a client query)
simulated_embedding = embedding = encoder.encode([test_text])[0]  # This is what gets intercepted

# 4. Attacker attempts reconstruction via cosine similarity
similarities = cosine_similarity([simulated_embedding], corpus_embeddings)
closest_idx = np.argmax(similarities)

# 5. Display result
print(f"❗ Intercepted embedding likely corresponds to:")
print(f"🔍 Closest match: \"{corpus_sentences[closest_idx]}\"")
print(f"📈 Similarity score: {similarities[0][closest_idx]:.4f}")



❗ Intercepted embedding likely corresponds to:
🔍 Closest match: "I feel tired and sad all the time."
📈 Similarity score: 0.7972


**🔐 Encrypted Inference with CKKS (TenSEAL)**

This version demonstrates inference over encrypted input using the CKKS homomorphic encryption scheme via the TenSEAL library.


*   This setup mimics a privacy-preserving inference service where sensitive user input remains encrypted throughout the prediction process—ideal for federated healthcare settings or scenarios involving untrusted compute infrastructure.
*   The input embedding (from MiniLM) is encrypted using CKKS before being sent to the model.
*   The logistic regression model performs a dot product and adds bias entirely in ciphertext space.
*   The encrypted result is decrypted only after computation, and the sigmoid function is applied client-side to produce a probability score.












In [28]:
# 5. Setup TenSEAL encryption context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.global_scale = 2 ** 40
context.generate_galois_keys()

# 6. Encrypted inference
test_text = "I feel hopeless and tired all the time."
#test_text = "I am good."
embedding = encoder.encode([test_text])[0]

encrypted_vec = ts.ckks_vector(context, embedding.tolist())

weights = model.coef_[0]
bias = model.intercept_[0]

# Compute encrypted dot product + bias
encrypted_result = encrypted_vec.dot(weights) + bias

# Decrypt and apply sigmoid
score = encrypted_result.decrypt()[0]
probability = 1 / (1 + np.exp(-score))

print(f"\nEncrypted prediction probability for:\n\"{test_text}\"\n→ {probability:.4f}")
print("✅ Risk Detected" if probability > 0.5 else "✅ No Risk Detected")


Encrypted prediction probability for:
"I feel hopeless and tired all the time."
→ 0.9036
✅ Risk Detected


**🛡️ Simulated Attacker Scenario: Why Homomorphic Encryption Prevents Data Leakage**

This section demonstrates how a malicious actor — even if they intercept the encrypted embedding or encrypted inference result — cannot extract any meaningful information.

Three common attack attempts are shown:

**🔓 Direct Inspection of Encrypted Vector**

The attacker tries to read or print the contents of the encrypted vector.
→ Fails: CKKS ciphertexts are opaque; raw values are hidden.

**🔍 Cosine Similarity Matching**

The attacker tries to compare the encrypted vector with known plaintext embeddings (e.g. via cosine similarity) to infer meaning.
→ Fails: Encrypted vectors are not compatible with NumPy or sklearn operations.

**🔑 Decryption Without Proper Context**

The attacker creates a new TenSEAL context and attempts to decrypt the ciphertext.
→ Fails: Decryption requires the original private key tied to the encryption context.

In [29]:
# 🚨 Simulated Attacker Section
print("\n🛑 Simulated Attacker Tries to Extract Information...")

try:
    print("🔓 Trying to read encrypted vector directly:")
    print(encrypted_vec)
except Exception as e:
    print("❌ Cannot read encrypted vector:", e)

try:
    print("\n🔍 Trying cosine similarity on encrypted vector:")
    from sklearn.metrics.pairwise import cosine_similarity
    cosine_similarity([encrypted_vec], [embedding])  # Invalid: encrypted_vec is not a NumPy array
except Exception as e:
    print("❌ Cosine similarity failed:", e)

try:
    print("\n🔑 Trying to decrypt without context (attacker):")
    fake_context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[60, 40, 40, 60]
    )
    fake_vec = ts.ckks_vector(fake_context, embedding.tolist())
    _ = fake_vec.decrypt()  # Decryption fails due to missing keys
except Exception as e:
    print("❌ Decryption failed (attacker has no private key):", e)



🛑 Simulated Attacker Tries to Extract Information...
🔓 Trying to read encrypted vector directly:

🔍 Trying cosine similarity on encrypted vector:
❌ Cosine similarity failed: float() argument must be a string or a real number, not 'CKKSVector'

🔑 Trying to decrypt without context (attacker):
❌ Decryption failed (attacker has no private key): no global scale
